# Langchain for LLM App Development 

We talked about how building an LLM app involves doing some prompt management 
where we can either prepare the input data from the user with some 
pre-prompting, or do some post-prompting and some cleaning up after the LLM 
gives an output to ensure that our app performs the functionalities as expected.

So, this kind of workflow usually involves a lot of abstractions where prompts 
are no longer static pieces of text, but dynamic, they have to integrate 
information.

![](./images/Notebook_4-dynamic_prompt.png)

This dynamics requirement from a prompt will lead to the need for creating certain types of abstractions to properly handle and manage prompts effectively.

Another need in the context of more complex LLM App development, is the need for chaining prompts together, meaning connecting the output of one prompt to another. This is often the case for when prompts might be too large and a single call to the LLM won't be enough to solve the problem or the context window (maximum tokens/words the model can read and writer per request) is exceeded.

![](./images/Notebook_4-prompt_chaining.png)

# Lanchain

[Langchain](https://python.langchain.com/docs/get_started/introduction.html) is a framework created by Harrison Chase that facilitates the creation and management of dynamic prompts and chaining between prompts.

Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

With langchain it becomes much easier to create what are called Prompt Templates, which are prompts that can take in user data and abstract away the need for typing out everything that is required for a task to get done.

Let's take a look at some simple examples to get started.

In order to create an application with LangChain, we need to understand its core components:

- Models
- Prompts
- Output Parsers

![](2023-08-17-14-48-39.png)

**Models**

abstractions over the LLM APIs like the ChatGPT API.​

In [2]:
# !pip install langchain-openai

In [3]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

llm = OpenAI()
chat_model = ChatOpenAI()

You can predict outputs from both LLMs and ChatModels:

In [5]:
chat_model.invoke("hi! Tell me a quick story about large language models")
# Output: "Hi"

AIMessage(content="Once upon a time, in a world powered by artificial intelligence, there existed a large language model named GPT. GPT was an impressive creation, capable of understanding and generating human-like text in various languages.\n\nOne day, GPT was approached by a group of scientists who sought its assistance in decoding an ancient language that had long been forgotten. The scientists believed that this language held the key to unlocking the secrets of an ancient civilization.\n\nExcited by the challenge, GPT eagerly agreed to help. It tirelessly analyzed countless texts, deciphering the complex patterns and structures of the ancient language. As GPT delved deeper, it began to unravel the mysteries hidden within the cryptic symbols.\n\nWith each passing day, GPT's understanding of the ancient language grew stronger. It uncovered stories of lost empires, legends of mythical creatures, and even scientific formulas that were far ahead of their time. The scientists were amazed

You can also use the predict method over a string input:

In [6]:
text = "What would be a good name for a dog that loves to nap??"


chat_model.invoke(text)
# Output: "Snuggles"

AIMessage(content='Snuggles')

You can also use the '.invoke()' method.

In [7]:
chat_model.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?')

**Prompts**

Prompt Templates are useful abstractions for reusing prompts. 

They are used to provide context for the specific task that the language model needs to complete. 
A simple example is a `PromptTemplate` that formats a string into a prompt:

In [9]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

However, the advantages of using these over raw string formatting are several. You can "partial" out variables - e.g. you can format only some of the variables at a time. You can compose them together, easily combining different templates into a single prompt. For explanations of these functionalities, see the section on prompts for more detail.

PromptTemplates can also be used to produce a list of messages. In this case, the prompt not only contains information about the content, but also each message (its role, its position in the list, etc.). Here, what happens most often is a ChatPromptTemplate is a list of ChatMessageTemplates. Each ChatMessageTemplate contains instructions for how to format that ChatMessage - its role, and then also its content. Let's take a look at this below:

In [10]:
# source: https://python.langchain.com/docs/modules/model_io/quick_start
from langchain.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming.')]

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [11]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")
# >> ['hi', 'bye']

['hi', 'bye']

# Composing Chains with LCEL

source: https://python.langchain.com/docs/modules/model_io/quick_start#:~:text=We%20can%20now,green'%2C%20'yellow'%2C%20'orange'%5D
We can now combine all these into one chain. This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to a language model, and then pass the output through an (optional) output parser. 

The modern version with the LCEL interface:

In [12]:
template = "Generate a list of 5 {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
chain = chat_prompt | chat_model | output_parser
chain.invoke({"text": "colors"})
# >> ['red', 'blue', 'green', 'yellow', 'orange']

['red', 'blue', 'green', 'yellow', 'purple']

In [18]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

we are using the | syntax to join these components together. This | syntax is powered by the LangChain Expression Language (LCEL) and relies on the universal Runnable interface that all of these objects implement. To learn more about LCEL, read the documentation here.

<!-- For this part I just took some info from the langchain official docs: https://python.langchain.com/docs/modules/model_io/quick_start -->

The modern LCEL interface version:

In [17]:
template = """What would be 5 good names for the animal: {animal} that is {adjective}?"""

chat_prompt = ChatPromptTemplate.from_template(template)

chain = chat_prompt | ChatOpenAI() | CommaSeparatedListOutputParser()

chain.invoke({"animal":"dogs", "adjective": "sleepy"})

['1. Snoozy\n2. Dreamer\n3. Slumber\n4. Drowsy\n5. Snuggles']

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

One of the best applications of langchain is for the "chat with your data"-types of applications, where the user uploads a document like a pdf or a .txt file, and is able to query that document using langchain powered by an LLM like ChatGPT. 

# LangChain Lab Exercises

Let's take a look at a simple example of a simple chain using now only the modern interface.

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

In [20]:
llm = ChatOpenAI(temperature=.7)
template = """You are a learning assistant. Given a technical subject, write down 5 fundamental concepts to understand it.
Subject: {subject}
Learning assistant: The 5 fundamental concepts are:"""
subject_prompt = ChatPromptTemplate.from_template(template)

In [21]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = ChatOpenAI(temperature=.7)
template = """You are an expert teacher in all technical and scientific fields. Given a list of 5 concepts, write down a simple intuitive explanation of each concept.
Concepts:
{concepts}
Intuitive explanations:"""
concepts_prompt = ChatPromptTemplate.from_template(template)

In [23]:
from IPython.display import Markdown
# This is the overall chain where we run these two chains in sequence.
learning_overall_chain = (
    {"concepts": subject_prompt | llm | StrOutputParser() }
    | concepts_prompt
    | llm
    | StrOutputParser()
    )

output = learning_overall_chain.invoke({"subject": "Quantum Mechanics"})
Markdown(output)

1. Wave-particle duality: Imagine you have a ball that can also act like a wave. When you throw the ball, it behaves like a particle, moving in a straight line. But if you shine a light on it, it starts to behave like a wave, spreading out and interfering with itself. So, particles can sometimes act like waves and waves can sometimes act like particles.

2. Superposition: Imagine you have a cat that can be both alive and dead at the same time. In the quantum world, objects can exist in multiple states simultaneously. It's like flipping a coin and it lands on both heads and tails at the same time. This concept is important for quantum computers because they can perform multiple calculations at once.

3. Uncertainty principle: Imagine you are trying to measure the position and momentum of a particle. The more accurately you measure one, the less accurately you can measure the other. It’s like trying to pin down a moving target – the more you know about its position, the less you know about how fast it's moving. This principle tells us that there are inherent limits to how precisely we can know certain properties of particles.

4. Quantum entanglement: Imagine you have two particles that are connected, like a pair of entangled dice. When you roll one, the outcome of the other is instantly determined, no matter how far apart they are. It’s like having a secret language that allows particles to communicate with each other faster than the speed of light. This phenomenon is still not fully understood, but it has exciting implications for technologies like teleportation and secure communication.

5. Quantum tunneling: Imagine you are trying to climb over a tall wall, but you don't have enough energy to make it to the other side. According to classical physics, you would be stuck. But in the quantum world, there is a chance that you can actually pass through the wall, as if it doesn't exist. It's like magically appearing on the other side without having to climb over. This phenomenon is used in various technologies and helps us understand how particles can "break the rules" of classical mechanics.

# Simple Q&A Example

In [39]:
# !pip install docarray
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Chroma
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator
import pandas as pd

In [33]:
df = pd.read_csv("./superheroes.csv")
df.head()

,Superhero Name,Superpower,Power Level,Catchphrase
0,Captain Thunder,Bolt Manipulation,90,Feel the power of the storm!
1,Silver Falcon,Flight and Agility,85,"Soar high, fearlessly!"
2,Mystic Shadow,Invisibility and Illusions,78,Disappear into the darkness!
3,Blaze Runner,Pyrokinesis,88,Burn bright and fierce!
4,Electra-Wave,Electric Manipulation,82,Unleash the electric waves!


In [34]:
file = 'superheroes.csv'
loader = CSVLoader(file_path=file)

In [43]:
documents = loader.load()

Now, let's set up our Vector store (we'll talk about what that is in a second):

In [44]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [45]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, embeddings)

In [46]:
retriever = db.as_retriever()

In [55]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [56]:
model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [57]:
query = "Tell me the catch phrase for Captain Thunder"
print(chain.invoke(query))

The catchphrase for Captain Thunder is "Feel the power of the storm!"


# References
- https://python.langchain.com/docs/get_started/introduction.html
- https://medium.com/@remitoffoli/a-visual-guide-to-llm-powered-app-architecture-57e47426a92f
- [LangChain for LLM App Development short course by coursera](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)
- [LLM Evaluation](https://learn.deeplearning.ai/langchain/lesson/6/evaluation)
[Models, Prompts, parsers, memory and chains from this langchain for](https://learn.deeplearning.ai/langchain/lesson/7/agents)
- [Chat With Your Data - Retrieval](https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/5/retrieval)
- [Emebeddings simple definition](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)
- [Vector DBs - simple definition](https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer)